<h1>Crackup</h1>

Our group goal was to pull out ATL03 and ATL06 segments from two areas where we saw crevasses, and compare small scale features in the two datasets


<h2>Location/time </h2>
After a reeliminary look at the data, We pulled out a granules at two locations using KML files to and a short time    window

<ul>
    <li>georgeVI_meltpond-polygon.kml : Feb5 - Feb7, 2019
    <li>edgeworth_simple-polygon.kml : Oct 14 - Oct 16, 2018
 </ul>


We tried a more complex polygon, and this program had trouble finding the data.  So we had to go with a simple polygon.

<h2>Downloading ATL03 data</h2>
The code we used to get the ATL03 data for a specific time period within a KML polygon is shown below.  It is broken up into two sections.

In [5]:
#import libraries
import requests
import getpass
import socket
import json
import zipfile
import io
import math
import os
import shutil
import pprint
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
import h5py
import re
# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
from statistics import mean
from requests.auth import HTTPBasicAuth

# Earthdata Login credentials

# Enter your Earthdata Login user name
uid = 'slhoward'
# Enter your email address associated with your Earthdata Login account
email = 'showard@esr.org'
pswd = getpass.getpass('Earthdata Login password: ')

# Request token from Common Metadata Repository using Earthdata credentials
token_api_url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': uid,
        'password': pswd,
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}
headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
token = json.loads(response.content)['token']['id']
print(token)

Earthdata Login password:  ·········


CA5BB146-1523-778D-77BE-5B5E97F678B8


<b>In the code below, set:
<ul> 
    <li>short_name 
    <li>temporal range
    <li>kml_filepath
    <li> output file directory.  Default is ./Output_files
</ul>
</b>   

In [8]:
# Input data set ID (e.g. ATL06) of interest here, also known as "short name".

short_name = 'ATL03'

# Get json response from CMR collection metadata and print results. This provides high-level metadata on a data set or "collection", provide in json format.

params = {
    'short_name': short_name
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)
#pprint.pprint(results)

# Find all instances of 'version_id' in metadata and print most recent version number

versions = [i['version_id'] for i in results['feed']['entry']]
latest_version = max(versions)
print(latest_version)

# Input temporal range 

# Input start date in yyyy-MM-dd format
start_date = '2019-02-05'
# Input start time in HH:mm:ss format
start_time = '00:00:00'
# Input end date in yyyy-MM-dd format
end_date = '2019-02-07'
# Input end time in HH:mm:ss format
end_time = '23:59:59'

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'
print(temporal)

# Use geopandas to read in polygon file
# Note: a shapefile or geojson, or almost any other vector-based spatial data format could be substituted here.

kml_filepath = str('/home/jovyan/crackup/shapefiles/georgeVI_meltpond-polygon.kml')
print(kml_filepath)
#Return a GeoDataFrame object
gdf = gpd.read_file(kml_filepath)

#Integer position based indexing of GeoDataFrame object to get it into a shapeply geometry object.
poly = gdf.iloc[0].geometry

# Simplify polygon. The larger the tolerance value, the more simplified the polygon.
poly = poly.simplify(0.05, preserve_topology=False)

# Orient counter-clockwise
poly = orient(poly, sign=1.0)

#print(poly)

#Format dictionary to polygon coordinate pairs for CMR polygon filtering
polygon = ','.join([str(c) for xy in zip(*poly.exterior.coords.xy) for c in xy])

# aoi value used for CMR params below
aoi = '3'


#Create CMR parameters used for granule search. Modify params depending on bounding_box or polygon input.

if aoi == '1':
# bounding box input:
    params = {
    'short_name': short_name,
    'version': latest_version,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1,
    'bounding_box': bounding_box
    }
else:
    
# If polygon input (either via coordinate pairs or shapefile/KML/KMZ):
    params = {
    'short_name': short_name,
    'version': latest_version,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1,
    'polygon': polygon,
    }

#print('CMR search parameters: ', params)

# Query number of granules using our (paging over results)

granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'

granules = []
while True:
    response = requests.get(granule_search_url, params=params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    params['page_num'] += 1

    
# Get number of granules over my area and time of interest
len(granules)

granule_sizes = [float(granule['granule_size']) for granule in granules]

# Average size of granules in MB
mean(granule_sizes)

# Total volume in MB
sum(granule_sizes)

# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

print(capability_url)

# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))

root = ET.fromstring(response.content)

# collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

# variable subsetting
variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
variables_raw = [variables[i]['value'] for i in range(len(variables))]
variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
variable_vals = [v.replace(':', '/') for v in variables_join]

# reformatting
formats = [Format.attrib for Format in root.iter('Format')]
format_vals = [formats[i]['value'] for i in range(len(formats))]
format_vals.remove('')

# reprojection only applicable on ICESat-2 L3B products, yet to be available. 

# reformatting options that support reprojection
normalproj = [Projections.attrib for Projections in root.iter('Projections')]
normalproj_vals = []
normalproj_vals.append(normalproj[0]['normalProj'])
format_proj = normalproj_vals[0].split(',')
format_proj.remove('')
format_proj.append('No reformatting')

#reprojection options
projections = [Projection.attrib for Projection in root.iter('Projection')]
proj_vals = []
for i in range(len(projections)):
    if (projections[i]['value']) != 'NO_CHANGE' :
        proj_vals.append(projections[i]['value'])
        
# reformatting options that do not support reprojection
no_proj = [i for i in format_vals if i not in format_proj]

print(subagent)
if len(subagent) < 1 :
    agent = 'NO'
    
# Temporal subsetting KVP

timevar = start_date + 'T' + start_time + ',' + end_date + 'T' + end_time
print(timevar) 

coverage = '/,\
//ds_surf_type,\
//ds_xyz,\
/ancillary_data,\
/ancillary_data/atl03_pad,\
/ancillary_data/atlas_sdp_gps_epoch,\
/ancillary_data/control,\
/ancillary_data/data_end_utc,\
/ancillary_data/data_start_utc,\
/ancillary_data/end_cycle,\
/ancillary_data/end_delta_time,\
/ancillary_data/end_geoseg,\
/ancillary_data/end_gpssow,\
/ancillary_data/end_gpsweek,\
/ancillary_data/end_orbit,\
/ancillary_data/end_region,\
/ancillary_data/end_rgt,\
/ancillary_data/granule_end_utc,\
/ancillary_data/granule_start_utc,\
/ancillary_data/podppd_pad,\
/ancillary_data/release,\
/ancillary_data/start_cycle,\
/ancillary_data/start_delta_time,\
/ancillary_data/start_geoseg,\
/ancillary_data/start_gpssow,\
/ancillary_data/start_gpsweek,\
/ancillary_data/start_orbit,\
/ancillary_data/start_region,\
/ancillary_data/start_rgt,\
/ancillary_data/version,\
/ancillary_data/atlas_engineering,\
/ancillary_data/atlas_engineering/ds_gt,\
/ancillary_data/atlas_engineering/ds_stat,\
/ancillary_data/atlas_engineering/ph_uncorrelated_error,\
/ancillary_data/atlas_engineering/receiver,\
/ancillary_data/atlas_engineering/receiver/rx_bckgrd_sensitivity,\
/ancillary_data/atlas_engineering/receiver/rx_return_sensitivity,\
/ancillary_data/atlas_engineering/transmit,\
/ancillary_data/atlas_engineering/transmit/tx_pulse_distribution,\
/ancillary_data/atlas_engineering/transmit/tx_pulse_energy,\
/ancillary_data/atlas_engineering/transmit/tx_pulse_skew_est,\
/ancillary_data/atlas_engineering/transmit/tx_pulse_thresh_lower,\
/ancillary_data/atlas_engineering/transmit/tx_pulse_thresh_upper,\
/ancillary_data/atlas_engineering/transmit/tx_pulse_width_lower,\
/ancillary_data/atlas_engineering/transmit/tx_pulse_width_upper,\
/ancillary_data/calibrations,\
/ancillary_data/calibrations/ds_channel,\
/ancillary_data/calibrations/dead_time,\
/ancillary_data/calibrations/dead_time/cal42_product,\
/ancillary_data/calibrations/dead_time/side,\
/ancillary_data/calibrations/dead_time/temperature,\
/ancillary_data/calibrations/dead_time/gt1l,\
/ancillary_data/calibrations/dead_time/gt1l,\
/ancillary_data/calibrations/dead_time/gt1r,\
/ancillary_data/calibrations/dead_time/gt1r,\
/ancillary_data/calibrations/dead_time/gt2l,\
/ancillary_data/calibrations/dead_time/gt2l,\
/ancillary_data/calibrations/dead_time/gt2r,\
/ancillary_data/calibrations/dead_time/gt2r,\
/ancillary_data/calibrations/dead_time/gt3l,\
/ancillary_data/calibrations/dead_time/gt3l,\
/ancillary_data/calibrations/dead_time/gt3r,\
/ancillary_data/calibrations/dead_time/gt3r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/cal34_product,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt1r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt2r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3l,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3r,\
/ancillary_data/calibrations/dead_time_radiometric_signal_loss/gt3r,\
/ancillary_data/calibrations/first_photon_bias,\
/ancillary_data/calibrations/first_photon_bias/cal19_product,\
/ancillary_data/calibrations/first_photon_bias/gt1l,\
/ancillary_data/calibrations/first_photon_bias/gt1l,\
/ancillary_data/calibrations/first_photon_bias/gt1l,\
/ancillary_data/calibrations/first_photon_bias/gt1l,\
/ancillary_data/calibrations/first_photon_bias/gt1r,\
/ancillary_data/calibrations/first_photon_bias/gt1r,\
/ancillary_data/calibrations/first_photon_bias/gt1r,\
/ancillary_data/calibrations/first_photon_bias/gt1r,\
/ancillary_data/calibrations/first_photon_bias/gt2l,\
/ancillary_data/calibrations/first_photon_bias/gt2l,\
/ancillary_data/calibrations/first_photon_bias/gt2l,\
/ancillary_data/calibrations/first_photon_bias/gt2l,\
/ancillary_data/calibrations/first_photon_bias/gt2r,\
/ancillary_data/calibrations/first_photon_bias/gt2r,\
/ancillary_data/calibrations/first_photon_bias/gt2r,\
/ancillary_data/calibrations/first_photon_bias/gt2r,\
/ancillary_data/calibrations/first_photon_bias/gt3l,\
/ancillary_data/calibrations/first_photon_bias/gt3l,\
/ancillary_data/calibrations/first_photon_bias/gt3l,\
/ancillary_data/calibrations/first_photon_bias/gt3l,\
/ancillary_data/calibrations/first_photon_bias/gt3r,\
/ancillary_data/calibrations/first_photon_bias/gt3r,\
/ancillary_data/calibrations/first_photon_bias/gt3r,\
/ancillary_data/calibrations/first_photon_bias/gt3r,\
/ancillary_data/calibrations/low_link_impulse_response,\
/ancillary_data/calibrations/low_link_impulse_response/bin_width,\
/ancillary_data/calibrations/low_link_impulse_response/cal20_product,\
/ancillary_data/calibrations/low_link_impulse_response/hist_x,\
/ancillary_data/calibrations/low_link_impulse_response/laser,\
/ancillary_data/calibrations/low_link_impulse_response/mode,\
/ancillary_data/calibrations/low_link_impulse_response/num_bins,\
/ancillary_data/calibrations/low_link_impulse_response/return_source,\
/ancillary_data/calibrations/low_link_impulse_response/side,\
/ancillary_data/calibrations/low_link_impulse_response/temperature,\
/ancillary_data/calibrations/low_link_impulse_response/gt1l,\
/ancillary_data/calibrations/low_link_impulse_response/gt1l,\
/ancillary_data/calibrations/low_link_impulse_response/gt1r,\
/ancillary_data/calibrations/low_link_impulse_response/gt1r,\
/ancillary_data/calibrations/low_link_impulse_response/gt2l,\
/ancillary_data/calibrations/low_link_impulse_response/gt2l,\
/ancillary_data/calibrations/low_link_impulse_response/gt2r,\
/ancillary_data/calibrations/low_link_impulse_response/gt2r,\
/ancillary_data/calibrations/low_link_impulse_response/gt3l,\
/ancillary_data/calibrations/low_link_impulse_response/gt3l,\
/ancillary_data/calibrations/low_link_impulse_response/gt3r,\
/ancillary_data/calibrations/low_link_impulse_response/gt3r,\
/ancillary_data/gt1l,\
/ancillary_data/gt1l/signal_find_input,\
/ancillary_data/gt1l/signal_find_input/addpad_flag,\
/ancillary_data/gt1l/signal_find_input/alpha_inc,\
/ancillary_data/gt1l/signal_find_input/alpha_max,\
/ancillary_data/gt1l/signal_find_input/delta_t_gap_min,\
/ancillary_data/gt1l/signal_find_input/delta_t_lin_fit,\
/ancillary_data/gt1l/signal_find_input/delta_t_max,\
/ancillary_data/gt1l/signal_find_input/delta_t_min,\
/ancillary_data/gt1l/signal_find_input/delta_z_bg,\
/ancillary_data/gt1l/signal_find_input/delta_zmax2,\
/ancillary_data/gt1l/signal_find_input/delta_zmin,\
/ancillary_data/gt1l/signal_find_input/e_a,\
/ancillary_data/gt1l/signal_find_input/e_linfit_edit,\
/ancillary_data/gt1l/signal_find_input/e_linfit_slant,\
/ancillary_data/gt1l/signal_find_input/e_m,\
/ancillary_data/gt1l/signal_find_input/e_m_mult,\
/ancillary_data/gt1l/signal_find_input/htspanmin,\
/ancillary_data/gt1l/signal_find_input/lslant_flag,\
/ancillary_data/gt1l/signal_find_input/min_fit_time_fact,\
/ancillary_data/gt1l/signal_find_input/nbin_min,\
/ancillary_data/gt1l/signal_find_input/n_delta_z1,\
/ancillary_data/gt1l/signal_find_input/n_delta_z2,\
/ancillary_data/gt1l/signal_find_input/nphot_min,\
/ancillary_data/gt1l/signal_find_input/nslw,\
/ancillary_data/gt1l/signal_find_input/nslw_v,\
/ancillary_data/gt1l/signal_find_input/out_edit_flag,\
/ancillary_data/gt1l/signal_find_input/pc_bckgrd_flag,\
/ancillary_data/gt1l/signal_find_input/r,\
/ancillary_data/gt1l/signal_find_input/r2,\
/ancillary_data/gt1l/signal_find_input/sig_find_t_inc,\
/ancillary_data/gt1l/signal_find_input/snrlow,\
/ancillary_data/gt1l/signal_find_input/snrmed,\
/ancillary_data/gt1l/signal_find_input/t_gap_big,\
/ancillary_data/gt1r,\
/ancillary_data/gt1r/signal_find_input,\
/ancillary_data/gt1r/signal_find_input/addpad_flag,\
/ancillary_data/gt1r/signal_find_input/alpha_inc,\
/ancillary_data/gt1r/signal_find_input/alpha_max,\
/ancillary_data/gt1r/signal_find_input/delta_t_gap_min,\
/ancillary_data/gt1r/signal_find_input/delta_t_lin_fit,\
/ancillary_data/gt1r/signal_find_input/delta_t_max,\
/ancillary_data/gt1r/signal_find_input/delta_t_min,\
/ancillary_data/gt1r/signal_find_input/delta_z_bg,\
/ancillary_data/gt1r/signal_find_input/delta_zmax2,\
/ancillary_data/gt1r/signal_find_input/delta_zmin,\
/ancillary_data/gt1r/signal_find_input/e_a,\
/ancillary_data/gt1r/signal_find_input/e_linfit_edit,\
/ancillary_data/gt1r/signal_find_input/e_linfit_slant,\
/ancillary_data/gt1r/signal_find_input/e_m,\
/ancillary_data/gt1r/signal_find_input/e_m_mult,\
/ancillary_data/gt1r/signal_find_input/htspanmin,\
/ancillary_data/gt1r/signal_find_input/lslant_flag,\
/ancillary_data/gt1r/signal_find_input/min_fit_time_fact,\
/ancillary_data/gt1r/signal_find_input/nbin_min,\
/ancillary_data/gt1r/signal_find_input/n_delta_z1,\
/ancillary_data/gt1r/signal_find_input/n_delta_z2,\
/ancillary_data/gt1r/signal_find_input/nphot_min,\
/ancillary_data/gt1r/signal_find_input/nslw,\
/ancillary_data/gt1r/signal_find_input/nslw_v,\
/ancillary_data/gt1r/signal_find_input/out_edit_flag,\
/ancillary_data/gt1r/signal_find_input/pc_bckgrd_flag,\
/ancillary_data/gt1r/signal_find_input/r,\
/ancillary_data/gt1r/signal_find_input/r2,\
/ancillary_data/gt1r/signal_find_input/sig_find_t_inc,\
/ancillary_data/gt1r/signal_find_input/snrlow,\
/ancillary_data/gt1r/signal_find_input/snrmed,\
/ancillary_data/gt1r/signal_find_input/t_gap_big,\
/ancillary_data/gt2l,\
/ancillary_data/gt2l/signal_find_input,\
/ancillary_data/gt2l/signal_find_input/addpad_flag,\
/ancillary_data/gt2l/signal_find_input/alpha_inc,\
/ancillary_data/gt2l/signal_find_input/alpha_max,\
/ancillary_data/gt2l/signal_find_input/delta_t_gap_min,\
/ancillary_data/gt2l/signal_find_input/delta_t_lin_fit,\
/ancillary_data/gt2l/signal_find_input/delta_t_max,\
/ancillary_data/gt2l/signal_find_input/delta_t_min,\
/ancillary_data/gt2l/signal_find_input/delta_z_bg,\
/ancillary_data/gt2l/signal_find_input/delta_zmax2,\
/ancillary_data/gt2l/signal_find_input/delta_zmin,\
/ancillary_data/gt2l/signal_find_input/e_a,\
/ancillary_data/gt2l/signal_find_input/e_linfit_edit,\
/ancillary_data/gt2l/signal_find_input/e_linfit_slant,\
/ancillary_data/gt2l/signal_find_input/e_m,\
/ancillary_data/gt2l/signal_find_input/e_m_mult,\
/ancillary_data/gt2l/signal_find_input/htspanmin,\
/ancillary_data/gt2l/signal_find_input/lslant_flag,\
/ancillary_data/gt2l/signal_find_input/min_fit_time_fact,\
/ancillary_data/gt2l/signal_find_input/nbin_min,\
/ancillary_data/gt2l/signal_find_input/n_delta_z1,\
/ancillary_data/gt2l/signal_find_input/n_delta_z2,\
/ancillary_data/gt2l/signal_find_input/nphot_min,\
/ancillary_data/gt2l/signal_find_input/nslw,\
/ancillary_data/gt2l/signal_find_input/nslw_v,\
/ancillary_data/gt2l/signal_find_input/out_edit_flag,\
/ancillary_data/gt2l/signal_find_input/pc_bckgrd_flag,\
/ancillary_data/gt2l/signal_find_input/r,\
/ancillary_data/gt2l/signal_find_input/r2,\
/ancillary_data/gt2l/signal_find_input/sig_find_t_inc,\
/ancillary_data/gt2l/signal_find_input/snrlow,\
/ancillary_data/gt2l/signal_find_input/snrmed,\
/ancillary_data/gt2l/signal_find_input/t_gap_big,\
/ancillary_data/gt2r,\
/ancillary_data/gt2r/signal_find_input,\
/ancillary_data/gt2r/signal_find_input/addpad_flag,\
/ancillary_data/gt2r/signal_find_input/alpha_inc,\
/ancillary_data/gt2r/signal_find_input/alpha_max,\
/ancillary_data/gt2r/signal_find_input/delta_t_gap_min,\
/ancillary_data/gt2r/signal_find_input/delta_t_lin_fit,\
/ancillary_data/gt2r/signal_find_input/delta_t_max,\
/ancillary_data/gt2r/signal_find_input/delta_t_min,\
/ancillary_data/gt2r/signal_find_input/delta_z_bg,\
/ancillary_data/gt2r/signal_find_input/delta_zmax2,\
/ancillary_data/gt2r/signal_find_input/delta_zmin,\
/ancillary_data/gt2r/signal_find_input/e_a,\
/ancillary_data/gt2r/signal_find_input/e_linfit_edit,\
/ancillary_data/gt2r/signal_find_input/e_linfit_slant,\
/ancillary_data/gt2r/signal_find_input/e_m,\
/ancillary_data/gt2r/signal_find_input/e_m_mult,\
/ancillary_data/gt2r/signal_find_input/htspanmin,\
/ancillary_data/gt2r/signal_find_input/lslant_flag,\
/ancillary_data/gt2r/signal_find_input/min_fit_time_fact,\
/ancillary_data/gt2r/signal_find_input/nbin_min,\
/ancillary_data/gt2r/signal_find_input/n_delta_z1,\
/ancillary_data/gt2r/signal_find_input/n_delta_z2,\
/ancillary_data/gt2r/signal_find_input/nphot_min,\
/ancillary_data/gt2r/signal_find_input/nslw,\
/ancillary_data/gt2r/signal_find_input/nslw_v,\
/ancillary_data/gt2r/signal_find_input/out_edit_flag,\
/ancillary_data/gt2r/signal_find_input/pc_bckgrd_flag,\
/ancillary_data/gt2r/signal_find_input/r,\
/ancillary_data/gt2r/signal_find_input/r2,\
/ancillary_data/gt2r/signal_find_input/sig_find_t_inc,\
/ancillary_data/gt2r/signal_find_input/snrlow,\
/ancillary_data/gt2r/signal_find_input/snrmed,\
/ancillary_data/gt2r/signal_find_input/t_gap_big,\
/ancillary_data/gt3l,\
/ancillary_data/gt3l/signal_find_input,\
/ancillary_data/gt3l/signal_find_input/addpad_flag,\
/ancillary_data/gt3l/signal_find_input/alpha_inc,\
/ancillary_data/gt3l/signal_find_input/alpha_max,\
/ancillary_data/gt3l/signal_find_input/delta_t_gap_min,\
/ancillary_data/gt3l/signal_find_input/delta_t_lin_fit,\
/ancillary_data/gt3l/signal_find_input/delta_t_max,\
/ancillary_data/gt3l/signal_find_input/delta_t_min,\
/ancillary_data/gt3l/signal_find_input/delta_z_bg,\
/ancillary_data/gt3l/signal_find_input/delta_zmax2,\
/ancillary_data/gt3l/signal_find_input/delta_zmin,\
/ancillary_data/gt3l/signal_find_input/e_a,\
/ancillary_data/gt3l/signal_find_input/e_linfit_edit,\
/ancillary_data/gt3l/signal_find_input/e_linfit_slant,\
/ancillary_data/gt3l/signal_find_input/e_m,\
/ancillary_data/gt3l/signal_find_input/e_m_mult,\
/ancillary_data/gt3l/signal_find_input/htspanmin,\
/ancillary_data/gt3l/signal_find_input/lslant_flag,\
/ancillary_data/gt3l/signal_find_input/min_fit_time_fact,\
/ancillary_data/gt3l/signal_find_input/nbin_min,\
/ancillary_data/gt3l/signal_find_input/n_delta_z1,\
/ancillary_data/gt3l/signal_find_input/n_delta_z2,\
/ancillary_data/gt3l/signal_find_input/nphot_min,\
/ancillary_data/gt3l/signal_find_input/nslw,\
/ancillary_data/gt3l/signal_find_input/nslw_v,\
/ancillary_data/gt3l/signal_find_input/out_edit_flag,\
/ancillary_data/gt3l/signal_find_input/pc_bckgrd_flag,\
/ancillary_data/gt3l/signal_find_input/r,\
/ancillary_data/gt3l/signal_find_input/r2,\
/ancillary_data/gt3l/signal_find_input/sig_find_t_inc,\
/ancillary_data/gt3l/signal_find_input/snrlow,\
/ancillary_data/gt3l/signal_find_input/snrmed,\
/ancillary_data/gt3l/signal_find_input/t_gap_big,\
/ancillary_data/gt3r,\
/ancillary_data/gt3r/signal_find_input,\
/ancillary_data/gt3r/signal_find_input/addpad_flag,\
/ancillary_data/gt3r/signal_find_input/alpha_inc,\
/ancillary_data/gt3r/signal_find_input/alpha_max,\
/ancillary_data/gt3r/signal_find_input/delta_t_gap_min,\
/ancillary_data/gt3r/signal_find_input/delta_t_lin_fit,\
/ancillary_data/gt3r/signal_find_input/delta_t_max,\
/ancillary_data/gt3r/signal_find_input/delta_t_min,\
/ancillary_data/gt3r/signal_find_input/delta_z_bg,\
/ancillary_data/gt3r/signal_find_input/delta_zmax2,\
/ancillary_data/gt3r/signal_find_input/delta_zmin,\
/ancillary_data/gt3r/signal_find_input/e_a,\
/ancillary_data/gt3r/signal_find_input/e_linfit_edit,\
/ancillary_data/gt3r/signal_find_input/e_linfit_slant,\
/ancillary_data/gt3r/signal_find_input/e_m,\
/ancillary_data/gt3r/signal_find_input/e_m_mult,\
/ancillary_data/gt3r/signal_find_input/htspanmin,\
/ancillary_data/gt3r/signal_find_input/lslant_flag,\
/ancillary_data/gt3r/signal_find_input/min_fit_time_fact,\
/ancillary_data/gt3r/signal_find_input/nbin_min,\
/ancillary_data/gt3r/signal_find_input/n_delta_z1,\
/ancillary_data/gt3r/signal_find_input/n_delta_z2,\
/ancillary_data/gt3r/signal_find_input/nphot_min,\
/ancillary_data/gt3r/signal_find_input/nslw,\
/ancillary_data/gt3r/signal_find_input/nslw_v,\
/ancillary_data/gt3r/signal_find_input/out_edit_flag,\
/ancillary_data/gt3r/signal_find_input/pc_bckgrd_flag,\
/ancillary_data/gt3r/signal_find_input/r,\
/ancillary_data/gt3r/signal_find_input/r2,\
/ancillary_data/gt3r/signal_find_input/sig_find_t_inc,\
/ancillary_data/gt3r/signal_find_input/snrlow,\
/ancillary_data/gt3r/signal_find_input/snrmed,\
/ancillary_data/gt3r/signal_find_input/t_gap_big,\
/ancillary_data/tep,\
/ancillary_data/tep/ds_gt,\
/ancillary_data/tep/min_tep_ph,\
/ancillary_data/tep/min_tep_secs,\
/ancillary_data/tep/n_tep_bins,\
/ancillary_data/tep/tep_bin_size,\
/ancillary_data/tep/tep_gap_size,\
/ancillary_data/tep/tep_normalize,\
/ancillary_data/tep/tep_peak_bins,\
/ancillary_data/tep/tep_prim_window,\
/ancillary_data/tep/tep_range_prim,\
/ancillary_data/tep/tep_rm_noise,\
/ancillary_data/tep/tep_sec_window,\
/ancillary_data/tep/tep_start_x,\
/ancillary_data/tep/tep_valid_spot,\
/atlas_impulse_response,\
/atlas_impulse_response/pce1_spot1,\
/atlas_impulse_response/pce1_spot1/tep_histogram,\
/atlas_impulse_response/pce1_spot1/tep_histogram/reference_tep_flag,\
/atlas_impulse_response/pce1_spot1/tep_histogram/tep_bckgrd,\
/atlas_impulse_response/pce1_spot1/tep_histogram/tep_duration,\
/atlas_impulse_response/pce1_spot1/tep_histogram/tep_hist,\
/atlas_impulse_response/pce1_spot1/tep_histogram/tep_hist_sum,\
/atlas_impulse_response/pce1_spot1/tep_histogram/tep_hist_time,\
/atlas_impulse_response/pce1_spot1/tep_histogram/tep_tod,\
/atlas_impulse_response/pce2_spot3,\
/atlas_impulse_response/pce2_spot3/tep_histogram,\
/atlas_impulse_response/pce2_spot3/tep_histogram/reference_tep_flag,\
/atlas_impulse_response/pce2_spot3/tep_histogram/tep_bckgrd,\
/atlas_impulse_response/pce2_spot3/tep_histogram/tep_duration,\
/atlas_impulse_response/pce2_spot3/tep_histogram/tep_hist,\
/atlas_impulse_response/pce2_spot3/tep_histogram/tep_hist_sum,\
/atlas_impulse_response/pce2_spot3/tep_histogram/tep_hist_time,\
/atlas_impulse_response/pce2_spot3/tep_histogram/tep_tod,\
/gt1l,\
/gt1l/bckgrd_atlas,\
/gt1l/bckgrd_atlas/bckgrd_counts,\
/gt1l/bckgrd_atlas/bckgrd_counts_reduced,\
/gt1l/bckgrd_atlas/bckgrd_hist_top,\
/gt1l/bckgrd_atlas/bckgrd_int_height,\
/gt1l/bckgrd_atlas/bckgrd_int_height_reduced,\
/gt1l/bckgrd_atlas/bckgrd_rate,\
/gt1l/bckgrd_atlas/delta_time,\
/gt1l/bckgrd_atlas/pce_mframe_cnt,\
/gt1l/bckgrd_atlas/tlm_height_band1,\
/gt1l/bckgrd_atlas/tlm_height_band2,\
/gt1l/bckgrd_atlas/tlm_top_band1,\
/gt1l/bckgrd_atlas/tlm_top_band2,\
/gt1l/geolocation,\
/gt1l/geolocation/altitude_sc,\
/gt1l/geolocation/bounce_time_offset,\
/gt1l/geolocation/delta_time,\
/gt1l/geolocation/neutat_delay_derivative,\
/gt1l/geolocation/neutat_delay_total,\
/gt1l/geolocation/neutat_ht,\
/gt1l/geolocation/ph_index_beg,\
/gt1l/geolocation/range_bias_corr,\
/gt1l/geolocation/ref_azimuth,\
/gt1l/geolocation/ref_elev,\
/gt1l/geolocation/reference_photon_index,\
/gt1l/geolocation/reference_photon_lat,\
/gt1l/geolocation/reference_photon_lon,\
/gt1l/geolocation/segment_dist_x,\
/gt1l/geolocation/segment_id,\
/gt1l/geolocation/segment_length,\
/gt1l/geolocation/segment_ph_cnt,\
/gt1l/geolocation/sigma_across,\
/gt1l/geolocation/sigma_along,\
/gt1l/geolocation/sigma_h,\
/gt1l/geolocation/sigma_lat,\
/gt1l/geolocation/sigma_lon,\
/gt1l/geolocation/solar_azimuth,\
/gt1l/geolocation/solar_elevation,\
/gt1l/geolocation/surf_type,\
/gt1l/geolocation/tx_pulse_energy,\
/gt1l/geolocation/tx_pulse_skew_est,\
/gt1l/geolocation/tx_pulse_width_lower,\
/gt1l/geolocation/tx_pulse_width_upper,\
/gt1l/geolocation/velocity_sc,\
/gt1l/geophys_corr,\
/gt1l/geophys_corr/dac,\
/gt1l/geophys_corr/delta_time,\
/gt1l/geophys_corr/geoid,\
/gt1l/geophys_corr/tide_earth,\
/gt1l/geophys_corr/tide_load,\
/gt1l/geophys_corr/tide_ocean,\
/gt1l/geophys_corr/tide_oc_pole,\
/gt1l/geophys_corr/tide_pole,\
/gt1l/heights,\
/gt1l/heights/delta_time,\
/gt1l/heights/dist_ph_across,\
/gt1l/heights/dist_ph_along,\
/gt1l/heights/h_ph,\
/gt1l/heights/lat_ph,\
/gt1l/heights/lon_ph,\
/gt1l/heights/pce_mframe_cnt,\
/gt1l/heights/ph_id_channel,\
/gt1l/heights/ph_id_count,\
/gt1l/heights/ph_id_pulse,\
/gt1l/heights/signal_conf_ph,\
/gt1l/signal_find_output,\
/gt1l/signal_find_output/inlandwater,\
/gt1l/signal_find_output/inlandwater/bckgrd_mean,\
/gt1l/signal_find_output/inlandwater/bckgrd_sigma,\
/gt1l/signal_find_output/inlandwater/delta_time,\
/gt1l/signal_find_output/inlandwater/t_pc_delta,\
/gt1l/signal_find_output/inlandwater/z_pc_delta,\
/gt1l/signal_find_output/land,\
/gt1l/signal_find_output/land/bckgrd_mean,\
/gt1l/signal_find_output/land/bckgrd_sigma,\
/gt1l/signal_find_output/land/delta_time,\
/gt1l/signal_find_output/land/t_pc_delta,\
/gt1l/signal_find_output/land/z_pc_delta,\
/gt1l/signal_find_output/land_ice,\
/gt1l/signal_find_output/land_ice/bckgrd_mean,\
/gt1l/signal_find_output/land_ice/bckgrd_sigma,\
/gt1l/signal_find_output/land_ice/delta_time,\
/gt1l/signal_find_output/land_ice/t_pc_delta,\
/gt1l/signal_find_output/land_ice/z_pc_delta,\
/gt1l/signal_find_output/ocean,\
/gt1l/signal_find_output/ocean/bckgrd_mean,\
/gt1l/signal_find_output/ocean/bckgrd_sigma,\
/gt1l/signal_find_output/ocean/delta_time,\
/gt1l/signal_find_output/ocean/t_pc_delta,\
/gt1l/signal_find_output/ocean/z_pc_delta,\
/gt1l/signal_find_output/sea_ice,\
/gt1l/signal_find_output/sea_ice/bckgrd_mean,\
/gt1l/signal_find_output/sea_ice/bckgrd_sigma,\
/gt1l/signal_find_output/sea_ice/delta_time,\
/gt1l/signal_find_output/sea_ice/t_pc_delta,\
/gt1l/signal_find_output/sea_ice/z_pc_delta,\
/gt1r,\
/gt1r/bckgrd_atlas,\
/gt1r/bckgrd_atlas/bckgrd_counts,\
/gt1r/bckgrd_atlas/bckgrd_counts_reduced,\
/gt1r/bckgrd_atlas/bckgrd_hist_top,\
/gt1r/bckgrd_atlas/bckgrd_int_height,\
/gt1r/bckgrd_atlas/bckgrd_int_height_reduced,\
/gt1r/bckgrd_atlas/bckgrd_rate,\
/gt1r/bckgrd_atlas/delta_time,\
/gt1r/bckgrd_atlas/pce_mframe_cnt,\
/gt1r/bckgrd_atlas/tlm_height_band1,\
/gt1r/bckgrd_atlas/tlm_height_band2,\
/gt1r/bckgrd_atlas/tlm_top_band1,\
/gt1r/bckgrd_atlas/tlm_top_band2,\
/gt1r/geolocation,\
/gt1r/geolocation/altitude_sc,\
/gt1r/geolocation/bounce_time_offset,\
/gt1r/geolocation/delta_time,\
/gt1r/geolocation/neutat_delay_derivative,\
/gt1r/geolocation/neutat_delay_total,\
/gt1r/geolocation/neutat_ht,\
/gt1r/geolocation/ph_index_beg,\
/gt1r/geolocation/range_bias_corr,\
/gt1r/geolocation/ref_azimuth,\
/gt1r/geolocation/ref_elev,\
/gt1r/geolocation/reference_photon_index,\
/gt1r/geolocation/reference_photon_lat,\
/gt1r/geolocation/reference_photon_lon,\
/gt1r/geolocation/segment_dist_x,\
/gt1r/geolocation/segment_id,\
/gt1r/geolocation/segment_length,\
/gt1r/geolocation/segment_ph_cnt,\
/gt1r/geolocation/sigma_across,\
/gt1r/geolocation/sigma_along,\
/gt1r/geolocation/sigma_h,\
/gt1r/geolocation/sigma_lat,\
/gt1r/geolocation/sigma_lon,\
/gt1r/geolocation/solar_azimuth,\
/gt1r/geolocation/solar_elevation,\
/gt1r/geolocation/surf_type,\
/gt1r/geolocation/tx_pulse_energy,\
/gt1r/geolocation/tx_pulse_skew_est,\
/gt1r/geolocation/tx_pulse_width_lower,\
/gt1r/geolocation/tx_pulse_width_upper,\
/gt1r/geolocation/velocity_sc,\
/gt1r/geophys_corr,\
/gt1r/geophys_corr/dac,\
/gt1r/geophys_corr/delta_time,\
/gt1r/geophys_corr/geoid,\
/gt1r/geophys_corr/tide_earth,\
/gt1r/geophys_corr/tide_load,\
/gt1r/geophys_corr/tide_ocean,\
/gt1r/geophys_corr/tide_oc_pole,\
/gt1r/geophys_corr/tide_pole,\
/gt1r/heights,\
/gt1r/heights/delta_time,\
/gt1r/heights/dist_ph_across,\
/gt1r/heights/dist_ph_along,\
/gt1r/heights/h_ph,\
/gt1r/heights/lat_ph,\
/gt1r/heights/lon_ph,\
/gt1r/heights/pce_mframe_cnt,\
/gt1r/heights/ph_id_channel,\
/gt1r/heights/ph_id_count,\
/gt1r/heights/ph_id_pulse,\
/gt1r/heights/signal_conf_ph,\
/gt1r/signal_find_output,\
/gt1r/signal_find_output/inlandwater,\
/gt1r/signal_find_output/inlandwater/bckgrd_mean,\
/gt1r/signal_find_output/inlandwater/bckgrd_sigma,\
/gt1r/signal_find_output/inlandwater/delta_time,\
/gt1r/signal_find_output/inlandwater/t_pc_delta,\
/gt1r/signal_find_output/inlandwater/z_pc_delta,\
/gt1r/signal_find_output/land,\
/gt1r/signal_find_output/land/bckgrd_mean,\
/gt1r/signal_find_output/land/bckgrd_sigma,\
/gt1r/signal_find_output/land/delta_time,\
/gt1r/signal_find_output/land/t_pc_delta,\
/gt1r/signal_find_output/land/z_pc_delta,\
/gt1r/signal_find_output/land_ice,\
/gt1r/signal_find_output/land_ice/bckgrd_mean,\
/gt1r/signal_find_output/land_ice/bckgrd_sigma,\
/gt1r/signal_find_output/land_ice/delta_time,\
/gt1r/signal_find_output/land_ice/t_pc_delta,\
/gt1r/signal_find_output/land_ice/z_pc_delta,\
/gt1r/signal_find_output/ocean,\
/gt1r/signal_find_output/ocean/bckgrd_mean,\
/gt1r/signal_find_output/ocean/bckgrd_sigma,\
/gt1r/signal_find_output/ocean/delta_time,\
/gt1r/signal_find_output/ocean/t_pc_delta,\
/gt1r/signal_find_output/ocean/z_pc_delta,\
/gt1r/signal_find_output/sea_ice,\
/gt1r/signal_find_output/sea_ice/bckgrd_mean,\
/gt1r/signal_find_output/sea_ice/bckgrd_sigma,\
/gt1r/signal_find_output/sea_ice/delta_time,\
/gt1r/signal_find_output/sea_ice/t_pc_delta,\
/gt1r/signal_find_output/sea_ice/z_pc_delta,\
/gt2l,\
/gt2l/bckgrd_atlas,\
/gt2l/bckgrd_atlas/bckgrd_counts,\
/gt2l/bckgrd_atlas/bckgrd_counts_reduced,\
/gt2l/bckgrd_atlas/bckgrd_hist_top,\
/gt2l/bckgrd_atlas/bckgrd_int_height,\
/gt2l/bckgrd_atlas/bckgrd_int_height_reduced,\
/gt2l/bckgrd_atlas/bckgrd_rate,\
/gt2l/bckgrd_atlas/delta_time,\
/gt2l/bckgrd_atlas/pce_mframe_cnt,\
/gt2l/bckgrd_atlas/tlm_height_band1,\
/gt2l/bckgrd_atlas/tlm_height_band2,\
/gt2l/bckgrd_atlas/tlm_top_band1,\
/gt2l/bckgrd_atlas/tlm_top_band2,\
/gt2l/geolocation,\
/gt2l/geolocation/altitude_sc,\
/gt2l/geolocation/bounce_time_offset,\
/gt2l/geolocation/delta_time,\
/gt2l/geolocation/neutat_delay_derivative,\
/gt2l/geolocation/neutat_delay_total,\
/gt2l/geolocation/neutat_ht,\
/gt2l/geolocation/ph_index_beg,\
/gt2l/geolocation/range_bias_corr,\
/gt2l/geolocation/ref_azimuth,\
/gt2l/geolocation/ref_elev,\
/gt2l/geolocation/reference_photon_index,\
/gt2l/geolocation/reference_photon_lat,\
/gt2l/geolocation/reference_photon_lon,\
/gt2l/geolocation/segment_dist_x,\
/gt2l/geolocation/segment_id,\
/gt2l/geolocation/segment_length,\
/gt2l/geolocation/segment_ph_cnt,\
/gt2l/geolocation/sigma_across,\
/gt2l/geolocation/sigma_along,\
/gt2l/geolocation/sigma_h,\
/gt2l/geolocation/sigma_lat,\
/gt2l/geolocation/sigma_lon,\
/gt2l/geolocation/solar_azimuth,\
/gt2l/geolocation/solar_elevation,\
/gt2l/geolocation/surf_type,\
/gt2l/geolocation/tx_pulse_energy,\
/gt2l/geolocation/tx_pulse_skew_est,\
/gt2l/geolocation/tx_pulse_width_lower,\
/gt2l/geolocation/tx_pulse_width_upper,\
/gt2l/geolocation/velocity_sc,\
/gt2l/geophys_corr,\
/gt2l/geophys_corr/dac,\
/gt2l/geophys_corr/delta_time,\
/gt2l/geophys_corr/geoid,\
/gt2l/geophys_corr/tide_earth,\
/gt2l/geophys_corr/tide_load,\
/gt2l/geophys_corr/tide_ocean,\
/gt2l/geophys_corr/tide_oc_pole,\
/gt2l/geophys_corr/tide_pole,\
/gt2l/heights,\
/gt2l/heights/delta_time,\
/gt2l/heights/dist_ph_across,\
/gt2l/heights/dist_ph_along,\
/gt2l/heights/h_ph,\
/gt2l/heights/lat_ph,\
/gt2l/heights/lon_ph,\
/gt2l/heights/pce_mframe_cnt,\
/gt2l/heights/ph_id_channel,\
/gt2l/heights/ph_id_count,\
/gt2l/heights/ph_id_pulse,\
/gt2l/heights/signal_conf_ph,\
/gt2l/signal_find_output,\
/gt2l/signal_find_output/inlandwater,\
/gt2l/signal_find_output/inlandwater/bckgrd_mean,\
/gt2l/signal_find_output/inlandwater/bckgrd_sigma,\
/gt2l/signal_find_output/inlandwater/delta_time,\
/gt2l/signal_find_output/inlandwater/t_pc_delta,\
/gt2l/signal_find_output/inlandwater/z_pc_delta,\
/gt2l/signal_find_output/land,\
/gt2l/signal_find_output/land/bckgrd_mean,\
/gt2l/signal_find_output/land/bckgrd_sigma,\
/gt2l/signal_find_output/land/delta_time,\
/gt2l/signal_find_output/land/t_pc_delta,\
/gt2l/signal_find_output/land/z_pc_delta,\
/gt2l/signal_find_output/land_ice,\
/gt2l/signal_find_output/land_ice/bckgrd_mean,\
/gt2l/signal_find_output/land_ice/bckgrd_sigma,\
/gt2l/signal_find_output/land_ice/delta_time,\
/gt2l/signal_find_output/land_ice/t_pc_delta,\
/gt2l/signal_find_output/land_ice/z_pc_delta,\
/gt2l/signal_find_output/ocean,\
/gt2l/signal_find_output/ocean/bckgrd_mean,\
/gt2l/signal_find_output/ocean/bckgrd_sigma,\
/gt2l/signal_find_output/ocean/delta_time,\
/gt2l/signal_find_output/ocean/t_pc_delta,\
/gt2l/signal_find_output/ocean/z_pc_delta,\
/gt2l/signal_find_output/sea_ice,\
/gt2l/signal_find_output/sea_ice/bckgrd_mean,\
/gt2l/signal_find_output/sea_ice/bckgrd_sigma,\
/gt2l/signal_find_output/sea_ice/delta_time,\
/gt2l/signal_find_output/sea_ice/t_pc_delta,\
/gt2l/signal_find_output/sea_ice/z_pc_delta,\
/gt2r,\
/gt2r/bckgrd_atlas,\
/gt2r/bckgrd_atlas/bckgrd_counts,\
/gt2r/bckgrd_atlas/bckgrd_counts_reduced,\
/gt2r/bckgrd_atlas/bckgrd_hist_top,\
/gt2r/bckgrd_atlas/bckgrd_int_height,\
/gt2r/bckgrd_atlas/bckgrd_int_height_reduced,\
/gt2r/bckgrd_atlas/bckgrd_rate,\
/gt2r/bckgrd_atlas/delta_time,\
/gt2r/bckgrd_atlas/pce_mframe_cnt,\
/gt2r/bckgrd_atlas/tlm_height_band1,\
/gt2r/bckgrd_atlas/tlm_height_band2,\
/gt2r/bckgrd_atlas/tlm_top_band1,\
/gt2r/bckgrd_atlas/tlm_top_band2,\
/gt2r/geolocation,\
/gt2r/geolocation/altitude_sc,\
/gt2r/geolocation/bounce_time_offset,\
/gt2r/geolocation/delta_time,\
/gt2r/geolocation/neutat_delay_derivative,\
/gt2r/geolocation/neutat_delay_total,\
/gt2r/geolocation/neutat_ht,\
/gt2r/geolocation/ph_index_beg,\
/gt2r/geolocation/range_bias_corr,\
/gt2r/geolocation/ref_azimuth,\
/gt2r/geolocation/ref_elev,\
/gt2r/geolocation/reference_photon_index,\
/gt2r/geolocation/reference_photon_lat,\
/gt2r/geolocation/reference_photon_lon,\
/gt2r/geolocation/segment_dist_x,\
/gt2r/geolocation/segment_id,\
/gt2r/geolocation/segment_length,\
/gt2r/geolocation/segment_ph_cnt,\
/gt2r/geolocation/sigma_across,\
/gt2r/geolocation/sigma_along,\
/gt2r/geolocation/sigma_h,\
/gt2r/geolocation/sigma_lat,\
/gt2r/geolocation/sigma_lon,\
/gt2r/geolocation/solar_azimuth,\
/gt2r/geolocation/solar_elevation,\
/gt2r/geolocation/surf_type,\
/gt2r/geolocation/tx_pulse_energy,\
/gt2r/geolocation/tx_pulse_skew_est,\
/gt2r/geolocation/tx_pulse_width_lower,\
/gt2r/geolocation/tx_pulse_width_upper,\
/gt2r/geolocation/velocity_sc,\
/gt2r/geophys_corr,\
/gt2r/geophys_corr/dac,\
/gt2r/geophys_corr/delta_time,\
/gt2r/geophys_corr/geoid,\
/gt2r/geophys_corr/tide_earth,\
/gt2r/geophys_corr/tide_load,\
/gt2r/geophys_corr/tide_ocean,\
/gt2r/geophys_corr/tide_oc_pole,\
/gt2r/geophys_corr/tide_pole,\
/gt2r/heights,\
/gt2r/heights/delta_time,\
/gt2r/heights/dist_ph_across,\
/gt2r/heights/dist_ph_along,\
/gt2r/heights/h_ph,\
/gt2r/heights/lat_ph,\
/gt2r/heights/lon_ph,\
/gt2r/heights/pce_mframe_cnt,\
/gt2r/heights/ph_id_channel,\
/gt2r/heights/ph_id_count,\
/gt2r/heights/ph_id_pulse,\
/gt2r/heights/signal_conf_ph,\
/gt2r/signal_find_output,\
/gt2r/signal_find_output/inlandwater,\
/gt2r/signal_find_output/inlandwater/bckgrd_mean,\
/gt2r/signal_find_output/inlandwater/bckgrd_sigma,\
/gt2r/signal_find_output/inlandwater/delta_time,\
/gt2r/signal_find_output/inlandwater/t_pc_delta,\
/gt2r/signal_find_output/inlandwater/z_pc_delta,\
/gt2r/signal_find_output/land,\
/gt2r/signal_find_output/land/bckgrd_mean,\
/gt2r/signal_find_output/land/bckgrd_sigma,\
/gt2r/signal_find_output/land/delta_time,\
/gt2r/signal_find_output/land/t_pc_delta,\
/gt2r/signal_find_output/land/z_pc_delta,\
/gt2r/signal_find_output/land_ice,\
/gt2r/signal_find_output/land_ice/bckgrd_mean,\
/gt2r/signal_find_output/land_ice/bckgrd_sigma,\
/gt2r/signal_find_output/land_ice/delta_time,\
/gt2r/signal_find_output/land_ice/t_pc_delta,\
/gt2r/signal_find_output/land_ice/z_pc_delta,\
/gt2r/signal_find_output/ocean,\
/gt2r/signal_find_output/ocean/bckgrd_mean,\
/gt2r/signal_find_output/ocean/bckgrd_sigma,\
/gt2r/signal_find_output/ocean/delta_time,\
/gt2r/signal_find_output/ocean/t_pc_delta,\
/gt2r/signal_find_output/ocean/z_pc_delta,\
/gt2r/signal_find_output/sea_ice,\
/gt2r/signal_find_output/sea_ice/bckgrd_mean,\
/gt2r/signal_find_output/sea_ice/bckgrd_sigma,\
/gt2r/signal_find_output/sea_ice/delta_time,\
/gt2r/signal_find_output/sea_ice/t_pc_delta,\
/gt2r/signal_find_output/sea_ice/z_pc_delta,\
/gt3l,\
/gt3l/bckgrd_atlas,\
/gt3l/bckgrd_atlas/bckgrd_counts,\
/gt3l/bckgrd_atlas/bckgrd_counts_reduced,\
/gt3l/bckgrd_atlas/bckgrd_hist_top,\
/gt3l/bckgrd_atlas/bckgrd_int_height,\
/gt3l/bckgrd_atlas/bckgrd_int_height_reduced,\
/gt3l/bckgrd_atlas/bckgrd_rate,\
/gt3l/bckgrd_atlas/delta_time,\
/gt3l/bckgrd_atlas/pce_mframe_cnt,\
/gt3l/bckgrd_atlas/tlm_height_band1,\
/gt3l/bckgrd_atlas/tlm_height_band2,\
/gt3l/bckgrd_atlas/tlm_top_band1,\
/gt3l/bckgrd_atlas/tlm_top_band2,\
/gt3l/geolocation,\
/gt3l/geolocation/altitude_sc,\
/gt3l/geolocation/bounce_time_offset,\
/gt3l/geolocation/delta_time,\
/gt3l/geolocation/neutat_delay_derivative,\
/gt3l/geolocation/neutat_delay_total,\
/gt3l/geolocation/neutat_ht,\
/gt3l/geolocation/ph_index_beg,\
/gt3l/geolocation/range_bias_corr,\
/gt3l/geolocation/ref_azimuth,\
/gt3l/geolocation/ref_elev,\
/gt3l/geolocation/reference_photon_index,\
/gt3l/geolocation/reference_photon_lat,\
/gt3l/geolocation/reference_photon_lon,\
/gt3l/geolocation/segment_dist_x,\
/gt3l/geolocation/segment_id,\
/gt3l/geolocation/segment_length,\
/gt3l/geolocation/segment_ph_cnt,\
/gt3l/geolocation/sigma_across,\
/gt3l/geolocation/sigma_along,\
/gt3l/geolocation/sigma_h,\
/gt3l/geolocation/sigma_lat,\
/gt3l/geolocation/sigma_lon,\
/gt3l/geolocation/solar_azimuth,\
/gt3l/geolocation/solar_elevation,\
/gt3l/geolocation/surf_type,\
/gt3l/geolocation/tx_pulse_energy,\
/gt3l/geolocation/tx_pulse_skew_est,\
/gt3l/geolocation/tx_pulse_width_lower,\
/gt3l/geolocation/tx_pulse_width_upper,\
/gt3l/geolocation/velocity_sc,\
/gt3l/geophys_corr,\
/gt3l/geophys_corr/dac,\
/gt3l/geophys_corr/delta_time,\
/gt3l/geophys_corr/geoid,\
/gt3l/geophys_corr/tide_earth,\
/gt3l/geophys_corr/tide_load,\
/gt3l/geophys_corr/tide_ocean,\
/gt3l/geophys_corr/tide_oc_pole,\
/gt3l/geophys_corr/tide_pole,\
/gt3l/heights,\
/gt3l/heights/delta_time,\
/gt3l/heights/dist_ph_across,\
/gt3l/heights/dist_ph_along,\
/gt3l/heights/h_ph,\
/gt3l/heights/lat_ph,\
/gt3l/heights/lon_ph,\
/gt3l/heights/pce_mframe_cnt,\
/gt3l/heights/ph_id_channel,\
/gt3l/heights/ph_id_count,\
/gt3l/heights/ph_id_pulse,\
/gt3l/heights/signal_conf_ph,\
/gt3l/signal_find_output,\
/gt3l/signal_find_output/inlandwater,\
/gt3l/signal_find_output/inlandwater/bckgrd_mean,\
/gt3l/signal_find_output/inlandwater/bckgrd_sigma,\
/gt3l/signal_find_output/inlandwater/delta_time,\
/gt3l/signal_find_output/inlandwater/t_pc_delta,\
/gt3l/signal_find_output/inlandwater/z_pc_delta,\
/gt3l/signal_find_output/land,\
/gt3l/signal_find_output/land/bckgrd_mean,\
/gt3l/signal_find_output/land/bckgrd_sigma,\
/gt3l/signal_find_output/land/delta_time,\
/gt3l/signal_find_output/land/t_pc_delta,\
/gt3l/signal_find_output/land/z_pc_delta,\
/gt3l/signal_find_output/land_ice,\
/gt3l/signal_find_output/land_ice/bckgrd_mean,\
/gt3l/signal_find_output/land_ice/bckgrd_sigma,\
/gt3l/signal_find_output/land_ice/delta_time,\
/gt3l/signal_find_output/land_ice/t_pc_delta,\
/gt3l/signal_find_output/land_ice/z_pc_delta,\
/gt3l/signal_find_output/ocean,\
/gt3l/signal_find_output/ocean/bckgrd_mean,\
/gt3l/signal_find_output/ocean/bckgrd_sigma,\
/gt3l/signal_find_output/ocean/delta_time,\
/gt3l/signal_find_output/ocean/t_pc_delta,\
/gt3l/signal_find_output/ocean/z_pc_delta,\
/gt3l/signal_find_output/sea_ice,\
/gt3l/signal_find_output/sea_ice/bckgrd_mean,\
/gt3l/signal_find_output/sea_ice/bckgrd_sigma,\
/gt3l/signal_find_output/sea_ice/delta_time,\
/gt3l/signal_find_output/sea_ice/t_pc_delta,\
/gt3l/signal_find_output/sea_ice/z_pc_delta,\
/gt3r,\
/gt3r/bckgrd_atlas,\
/gt3r/bckgrd_atlas/bckgrd_counts,\
/gt3r/bckgrd_atlas/bckgrd_counts_reduced,\
/gt3r/bckgrd_atlas/bckgrd_hist_top,\
/gt3r/bckgrd_atlas/bckgrd_int_height,\
/gt3r/bckgrd_atlas/bckgrd_int_height_reduced,\
/gt3r/bckgrd_atlas/bckgrd_rate,\
/gt3r/bckgrd_atlas/delta_time,\
/gt3r/bckgrd_atlas/pce_mframe_cnt,\
/gt3r/bckgrd_atlas/tlm_height_band1,\
/gt3r/bckgrd_atlas/tlm_height_band2,\
/gt3r/bckgrd_atlas/tlm_top_band1,\
/gt3r/bckgrd_atlas/tlm_top_band2,\
/gt3r/geolocation,\
/gt3r/geolocation/altitude_sc,\
/gt3r/geolocation/bounce_time_offset,\
/gt3r/geolocation/delta_time,\
/gt3r/geolocation/neutat_delay_derivative,\
/gt3r/geolocation/neutat_delay_total,\
/gt3r/geolocation/neutat_ht,\
/gt3r/geolocation/ph_index_beg,\
/gt3r/geolocation/range_bias_corr,\
/gt3r/geolocation/ref_azimuth,\
/gt3r/geolocation/ref_elev,\
/gt3r/geolocation/reference_photon_index,\
/gt3r/geolocation/reference_photon_lat,\
/gt3r/geolocation/reference_photon_lon,\
/gt3r/geolocation/segment_dist_x,\
/gt3r/geolocation/segment_id,\
/gt3r/geolocation/segment_length,\
/gt3r/geolocation/segment_ph_cnt,\
/gt3r/geolocation/sigma_across,\
/gt3r/geolocation/sigma_along,\
/gt3r/geolocation/sigma_h,\
/gt3r/geolocation/sigma_lat,\
/gt3r/geolocation/sigma_lon,\
/gt3r/geolocation/solar_azimuth,\
/gt3r/geolocation/solar_elevation,\
/gt3r/geolocation/surf_type,\
/gt3r/geolocation/tx_pulse_energy,\
/gt3r/geolocation/tx_pulse_skew_est,\
/gt3r/geolocation/tx_pulse_width_lower,\
/gt3r/geolocation/tx_pulse_width_upper,\
/gt3r/geolocation/velocity_sc,\
/gt3r/geophys_corr,\
/gt3r/geophys_corr/dac,\
/gt3r/geophys_corr/delta_time,\
/gt3r/geophys_corr/geoid,\
/gt3r/geophys_corr/tide_earth,\
/gt3r/geophys_corr/tide_load,\
/gt3r/geophys_corr/tide_ocean,\
/gt3r/geophys_corr/tide_oc_pole,\
/gt3r/geophys_corr/tide_pole,\
/gt3r/heights,\
/gt3r/heights/delta_time,\
/gt3r/heights/dist_ph_across,\
/gt3r/heights/dist_ph_along,\
/gt3r/heights/h_ph,\
/gt3r/heights/lat_ph,\
/gt3r/heights/lon_ph,\
/gt3r/heights/pce_mframe_cnt,\
/gt3r/heights/ph_id_channel,\
/gt3r/heights/ph_id_count,\
/gt3r/heights/ph_id_pulse,\
/gt3r/heights/signal_conf_ph,\
/gt3r/signal_find_output,\
/gt3r/signal_find_output/inlandwater,\
/gt3r/signal_find_output/inlandwater/bckgrd_mean,\
/gt3r/signal_find_output/inlandwater/bckgrd_sigma,\
/gt3r/signal_find_output/inlandwater/delta_time,\
/gt3r/signal_find_output/inlandwater/t_pc_delta,\
/gt3r/signal_find_output/inlandwater/z_pc_delta,\
/gt3r/signal_find_output/land,\
/gt3r/signal_find_output/land/bckgrd_mean,\
/gt3r/signal_find_output/land/bckgrd_sigma,\
/gt3r/signal_find_output/land/delta_time,\
/gt3r/signal_find_output/land/t_pc_delta,\
/gt3r/signal_find_output/land/z_pc_delta,\
/gt3r/signal_find_output/land_ice,\
/gt3r/signal_find_output/land_ice/bckgrd_mean,\
/gt3r/signal_find_output/land_ice/bckgrd_sigma,\
/gt3r/signal_find_output/land_ice/delta_time,\
/gt3r/signal_find_output/land_ice/t_pc_delta,\
/gt3r/signal_find_output/land_ice/z_pc_delta,\
/gt3r/signal_find_output/ocean,\
/gt3r/signal_find_output/ocean/bckgrd_mean,\
/gt3r/signal_find_output/ocean/bckgrd_sigma,\
/gt3r/signal_find_output/ocean/delta_time,\
/gt3r/signal_find_output/ocean/t_pc_delta,\
/gt3r/signal_find_output/ocean/z_pc_delta,\
/gt3r/signal_find_output/sea_ice,\
/gt3r/signal_find_output/sea_ice/bckgrd_mean,\
/gt3r/signal_find_output/sea_ice/bckgrd_sigma,\
/gt3r/signal_find_output/sea_ice/delta_time,\
/gt3r/signal_find_output/sea_ice/t_pc_delta,\
/gt3r/signal_find_output/sea_ice/z_pc_delta,\
/orbit_info,\
/orbit_info/crossing_time,\
/orbit_info/cycle_number,\
/orbit_info/lan,\
/orbit_info/orbit_number,\
/orbit_info/rgt,\
/orbit_info/sc_orient,\
/orbit_info/sc_orient_time,\
/quality_assessment,\
/quality_assessment/delta_time,\
/quality_assessment/qa_granule_fail_reason,\
/quality_assessment/qa_granule_pass_fail,\
/quality_assessment/gt1l,\
/quality_assessment/gt1l/qa_perc_signal_conf_ph_high,\
/quality_assessment/gt1l/qa_perc_signal_conf_ph_low,\
/quality_assessment/gt1l/qa_perc_signal_conf_ph_med,\
/quality_assessment/gt1l/qa_perc_surf_type,\
/quality_assessment/gt1l/qa_total_signal_conf_ph_high,\
/quality_assessment/gt1l/qa_total_signal_conf_ph_low,\
/quality_assessment/gt1l/qa_total_signal_conf_ph_med,\
/quality_assessment/gt1r,\
/quality_assessment/gt1r/qa_perc_signal_conf_ph_high,\
/quality_assessment/gt1r/qa_perc_signal_conf_ph_low,\
/quality_assessment/gt1r/qa_perc_signal_conf_ph_med,\
/quality_assessment/gt1r/qa_perc_surf_type,\
/quality_assessment/gt1r/qa_total_signal_conf_ph_high,\
/quality_assessment/gt1r/qa_total_signal_conf_ph_low,\
/quality_assessment/gt1r/qa_total_signal_conf_ph_med,\
/quality_assessment/gt2l,\
/quality_assessment/gt2l/qa_perc_signal_conf_ph_high,\
/quality_assessment/gt2l/qa_perc_signal_conf_ph_low,\
/quality_assessment/gt2l/qa_perc_signal_conf_ph_med,\
/quality_assessment/gt2l/qa_perc_surf_type,\
/quality_assessment/gt2l/qa_total_signal_conf_ph_high,\
/quality_assessment/gt2l/qa_total_signal_conf_ph_low,\
/quality_assessment/gt2l/qa_total_signal_conf_ph_med,\
/quality_assessment/gt2r,\
/quality_assessment/gt2r/qa_perc_signal_conf_ph_high,\
/quality_assessment/gt2r/qa_perc_signal_conf_ph_low,\
/quality_assessment/gt2r/qa_perc_signal_conf_ph_med,\
/quality_assessment/gt2r/qa_perc_surf_type,\
/quality_assessment/gt2r/qa_total_signal_conf_ph_high,\
/quality_assessment/gt2r/qa_total_signal_conf_ph_low,\
/quality_assessment/gt2r/qa_total_signal_conf_ph_med,\
/quality_assessment/gt3l,\
/quality_assessment/gt3l/qa_perc_signal_conf_ph_high,\
/quality_assessment/gt3l/qa_perc_signal_conf_ph_low,\
/quality_assessment/gt3l/qa_perc_signal_conf_ph_med,\
/quality_assessment/gt3l/qa_perc_surf_type,\
/quality_assessment/gt3l/qa_total_signal_conf_ph_high,\
/quality_assessment/gt3l/qa_total_signal_conf_ph_low,\
/quality_assessment/gt3l/qa_total_signal_conf_ph_med,\
/quality_assessment/gt3r,\
/quality_assessment/gt3r/qa_perc_signal_conf_ph_high,\
/quality_assessment/gt3r/qa_perc_signal_conf_ph_low,\
/quality_assessment/gt3r/qa_perc_signal_conf_ph_med,\
/quality_assessment/gt3r/qa_perc_surf_type,\
/quality_assessment/gt3r/qa_total_signal_conf_ph_high,\
/quality_assessment/gt3r/qa_total_signal_conf_ph_low,\
/quality_assessment/gt3r/qa_total_signal_conf_ph_med'


#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

# Set number of granules requested per order, which we will initially set to 10.
page_size = 10

#Determine number of pages basd on page_size and total granules. Loop requests by this value
page_num = math.ceil(len(granules)/page_size)

#Set request mode. 
request_mode = 'async'

# Determine how many individual orders we will request based on the number of granules requested

print(page_num)

API_request = f'{base_url}?short_name={short_name}&version={latest_version}&temporal={temporal}&time={timevar}&polygon={polygon}&Coverage={coverage}&request_mode={request_mode}&page_size={page_size}&page_num={page_num}&token={token}&email={email}'


subset_params = {
    'short_name': short_name, 
    'version': latest_version, 
    'temporal': temporal, 
    'time': timevar, 
    'polygon': polygon, 
#    'Coverage': coverage, 
    'request_mode': request_mode, 
    'page_size': page_size,  
    'token': token, 
    'email': email, 
    }
#print(subset_params)

path = str(os.getcwd() + '/Output_files')
if not os.path.exists(path):
    os.mkdir(path)
    
# Request data service for each page number, and unzip outputs

for i in range(page_num):
    page_val = i + 1
    print('Order: ', page_val)
    subset_params.update( {'page_num': page_val} )
    
# Post polygon to API endpoint for polygon subsetting to subset based on original, non-simplified KML file

    shape_post = {'shapefile': open(kml_filepath, 'rb')}
    request = session.post(base_url, params=subset_params, files=shape_post) 
    
# FOR ALL OTHER REQUESTS THAT DO NOT UTILIZED AN UPLOADED POLYGON FILE, USE A GET REQUEST INSTEAD OF POST:
#     request = session.get(base_url, params=request_params)
    
    print('Request HTTP response: ', request.status_code)

# Raise bad request: Loop will stop for bad response code.
    request.raise_for_status()
#    print('Order request URL: ', request.url)
    esir_root = ET.fromstring(request.content)
#    print('Order request response XML content: ', request.content)

# Look up order ID
    orderlist = []   
    for order in esir_root.findall("./order/"):
        orderlist.append(order.text)
    orderID = orderlist[0]
    print('order ID: ', orderID)

# Create status URL
    statusURL = base_url + '/' + orderID
    print('status URL: ', statusURL)

# Find order status
    request_response = session.get(statusURL)    
    print('HTTP response from order response URL: ', request_response.status_code)
    
# Raise bad request: Loop will stop for bad response code.
    request_response.raise_for_status()
    request_root = ET.fromstring(request_response.content)
    statuslist = []
    for status in request_root.findall("./requestStatus/"):
        statuslist.append(status.text)
    status = statuslist[0]
    print('Data request ', page_val, ' is submitting...')
    print('Initial request status is ', status)

# Continue to loop while request is still processing
    while status == 'pending' or status == 'processing': 
        print('Status is not complete. Trying again.')
        time.sleep(10)
        loop_response = session.get(statusURL)

# Raise bad request: Loop will stop for bad response code.
        loop_response.raise_for_status()
        loop_root = ET.fromstring(loop_response.content)

# Find status
        statuslist = []
        for status in loop_root.findall("./requestStatus/"):
            statuslist.append(status.text)
        status = statuslist[0]
        print('Retry request status is: ', status)
        if status == 'pending' or status == 'processing':
            continue

# Order can either complete, complete_with_errors, or fail:
# Provide complete_with_errors error message:
    if status == 'complete_with_errors' or status == 'failed':
        messagelist = []
        for message in loop_root.findall("./processInfo/"):
            messagelist.append(message.text)
        print('error messages:')
        pprint.pprint(messagelist)

# Download zipped order if status is complete or complete_with_errors
    if status == 'complete' or status == 'complete_with_errors':
        downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/' + orderID + '.zip'
        print('Zip download URL: ', downloadURL)
        print('Beginning download of zipped output...')
        zip_response = session.get(downloadURL)
        # Raise bad request: Loop will stop for bad response code.
        zip_response.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
            z.extractall(path)
        print('Data request', page_val, 'is complete.')
    else: print('Request failed.')

#Clean up Outputs folder by removing individual granule folders 

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
        
for root, dirs, files in os.walk(path):
    for name in dirs:
        os.rmdir(os.path.join(root, name))
        
#List files
sorted(os.listdir(path))

001
2019-02-05T00:00:00Z,2019-02-07T23:59:59Z
/home/jovyan/crackup/shapefiles/georgeVI_meltpond-polygon.kml
https://n5eil02u.ecs.nsidc.org/egi/capabilities/ATL03.001.xml
[{'id': 'ICESAT2', 'spatialSubsetting': 'true', 'spatialSubsettingShapefile': 'true', 'temporalSubsetting': 'true', 'type': 'both', 'maxGransSyncRequest': '100', 'maxGransAsyncRequest': '2000'}]
2019-02-05T00:00:00,2019-02-07T23:59:59
1
Order:  1
Request HTTP response:  201
order ID:  5000000320251
status URL:  https://n5eil02u.ecs.nsidc.org/egi/request/5000000320251
HTTP response from order response URL:  201
Data request  1  is submitting...
Initial request status is  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry request status is:  processing
Status is not complete. Trying again.
Retry 

['processed_ATL03_20190206021109_06050210_001_01.h5',
 'processed_ATL03_20190206145852_06130212_001_01.h5']

<h2>Downloading ATL06 data</h2>
Downloading the data was slightly different than the ATL03 data.  For this, we did not have to define the collections. The   It is broken up into two sections, as well.

In [12]:
import requests
import getpass
import socket
import json
import zipfile
import io
import math
import os
import shutil
import pprint
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
import h5py
import re
# To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
from statistics import mean
from requests.auth import HTTPBasicAuth

# Earthdata Login credentials

# Enter your Earthdata Login user name
uid = 'slhoward'
# Enter your email address associated with your Earthdata Login account
email = 'showard@esr.org'
pswd = getpass.getpass('Earthdata Login password: ')

# Request token from Common Metadata Repository using Earthdata credentials
token_api_url = 'https://cmr.earthdata.nasa.gov/legacy-services/rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': uid,
        'password': pswd,
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}
headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
token = json.loads(response.content)['token']['id']
print(token)
#4351E4A7-9907-AEA1-82B3-FE2A36C88659

Earthdata Login password:  ·········


A0F662C8-CF3F-EA6C-D4F3-706D22B72079


<b>In the code below, set:
<ul> 
    <li>short_name - set to ATL06
    <li>temporal range
    <li>kml_filepath
    <li> output file directory.  Default is ./Output_files
</ul>
        

In [13]:
# Input data set ID (e.g. ATL06) of interest here, also known as "short name".

short_name = 'ATL06'

# Get json response from CMR collection metadata and print results. This provides high-level metadata on a data set or "collection", provide in json format.

params = {
    'short_name': short_name
}

cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)
#pprint.pprint(results)

# Find all instances of 'version_id' in metadata and print most recent version number

versions = [i['version_id'] for i in results['feed']['entry']]
latest_version = max(versions)
print(latest_version)

# Input temporal range 

# Input start date in yyyy-MM-dd format
start_date = '2018-10-14'
# Input start time in HH:mm:ss format
start_time = '00:00:00'
# Input end date in yyyy-MM-dd format
end_date = '2018-10-16'
# Input end time in HH:mm:ss format
end_time = '23:59:59'

temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'
print(temporal)

# Use geopandas to read in polygon file
# Note: a shapefile or geojson, or almost any other vector-based spatial data format could be substituted here.

kml_filepath = str('/home/jovyan/crackup/shapefiles/edgeworth_simple-polygon.kml')
print(kml_filepath)

#Return a GeoDataFrame object
gdf = gpd.read_file(kml_filepath)
#gdf.head()

#Integer position based indexing of GeoDataFrame object to get it into a shapeply geometry object.
poly = gdf.iloc[0].geometry

# Simplify polygon. The larger the tolerance value, the more simplified the polygon.
poly = poly.simplify(0.05, preserve_topology=False)

# Orient counter-clockwise
poly = orient(poly, sign=1.0)

#print(poly)

#Format dictionary to polygon coordinate pairs for CMR polygon filtering
polygon = ','.join([str(c) for xy in zip(*poly.exterior.coords.xy) for c in xy])

# aoi value used for CMR params below
aoi = '3'

# Alternative option for spatial file input: Post file to OGR service
# Spatial file input, including Esri Shapefile or KML/KMZ
    
# POST shapefile or KML polygon to OGR for geojson conversion
url = 'http://ogre.adc4gis.com/convert'
shapefile = kml_filepath
files = {'upload': open(shapefile, 'rb')}
r = requests.post(url, files=files)
results = json.loads(r.content)
# Results is a dictionary representing a feature collection. List coordinates from the Polygon feature:
polygon_list = list(results['features'][0]['geometry']['coordinates'][0])     
# Remove z value from polygon list
for i in range(len(polygon_list)):
    del polygon_list[i][2] 
# Create shapely Polygon object for simplification and counter-clockwise ordering for CMR filtering
poly = Polygon(tuple(polygon_list))

#Same simplify and orient steps as above:
#simplify polygon
poly = poly.simplify(0.05, preserve_topology=False)

# Orient counter-clockwise
poly = orient(poly, sign=1.0)

#Format dictionary to polygon coordinate pairs for CMR polygon filtering
# Polygon points need to be provided in counter-clockwise order as comma separated values in longitude latitude order, i.e. lon1, lat1, lon2, lat2, lon3, lat3, and so on. 
# The last point should match the first point to close the polygon. 
polygon = ','.join([str(c) for xy in zip(*poly.exterior.coords.xy) for c in xy])

# aoi value used for subsetting logic below
aoi = '3'

#Create CMR parameters used for granule search. Modify params depending on bounding_box or polygon input.

if aoi == '1':
# bounding box input:
    params = {
    'short_name': short_name,
    'version': latest_version,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1,
    'bounding_box': bounding_box
    }
else:
    
# If polygon input (either via coordinate pairs or shapefile/KML/KMZ):
    params = {
    'short_name': short_name,
    'version': latest_version,
    'temporal': temporal,
    'page_size': 100,
    'page_num': 1,
    'polygon': polygon,
    }

    # Query number of granules using our (paging over results)

granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'

granules = []
while True:
    response = requests.get(granule_search_url, params=params, headers=headers)
    results = json.loads(response.content)

    if len(results['feed']['entry']) == 0:
        # Out of results, so break out of loop
        break

    # Collect results and increment page_num
    granules.extend(results['feed']['entry'])
    params['page_num'] += 1

    
# Get number of granules over my area and time of interest
len(granules)

granule_sizes = [float(granule['granule_size']) for granule in granules]

# Average size of granules in MB
mean(granule_sizes)

# Total volume in MB
sum(granule_sizes)

# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

print(capability_url)

# Create session to store cookie and pass credentials to capabilities url

session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(uid,pswd))

root = ET.fromstring(response.content)

# collect lists with each service option

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

# variable subsetting
variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
variables_raw = [variables[i]['value'] for i in range(len(variables))]
variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
variable_vals = [v.replace(':', '/') for v in variables_join]

# reformatting
formats = [Format.attrib for Format in root.iter('Format')]
format_vals = [formats[i]['value'] for i in range(len(formats))]
format_vals.remove('')

# reprojection only applicable on ICESat-2 L3B products, yet to be available. 

# reformatting options that support reprojection
normalproj = [Projections.attrib for Projections in root.iter('Projections')]
normalproj_vals = []
normalproj_vals.append(normalproj[0]['normalProj'])
format_proj = normalproj_vals[0].split(',')
format_proj.remove('')
format_proj.append('No reformatting')

#reprojection options
projections = [Projection.attrib for Projection in root.iter('Projection')]
proj_vals = []
for i in range(len(projections)):
    if (projections[i]['value']) != 'NO_CHANGE' :
        proj_vals.append(projections[i]['value'])
        
# reformatting options that do not support reprojection
no_proj = [i for i in format_vals if i not in format_proj]

print(subagent)
if len(subagent) < 1 :
    agent = 'NO'
    
# Temporal subsetting KVP

timevar = start_date + 'T' + start_time + ',' + end_date + 'T' + end_time
print(timevar)


#Set NSIDC data access base URL
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

# Set number of granules requested per order, which we will initially set to 10.
page_size = 10

#Determine number of pages basd on page_size and total granules. Loop requests by this value
page_num = math.ceil(len(granules)/page_size)

#Set request mode. 
request_mode = 'async'

# Determine how many individual orders we will request based on the number of granules requested

print(page_num)

subset_params = {
    'short_name': short_name, 
    'version': latest_version, 
    'temporal': temporal, 
    'time': timevar, 
    'polygon': polygon, 
#    'Coverage': coverage, 
    'request_mode': request_mode, 
    'page_size': page_size,  
    'token': token, 
    'email': email, 
    }

request_params = {
    'short_name': short_name, 
    'version': latest_version, 
    'temporal': temporal, 
    'polygon': polygon, 
    'agent' : 'NO',
    'include_meta' : 'Y',
    'request_mode': request_mode, 
    'page_size': page_size,  
    'token': token, 
    'email': email, 
    }

path = str(os.getcwd() + '/Output_files')
if not os.path.exists(path):
    os.mkdir(path)
    
# Request data service for each page number, and unzip outputs

for i in range(page_num):
    page_val = i + 1
    print('Order: ', page_val)
    subset_params.update( {'page_num': page_val} )
    
# Post polygon to API endpoint for polygon subsetting to subset based on original, non-simplified KML file

    shape_post = {'shapefile': open(kml_filepath, 'rb')}
    request = session.post(base_url, params=subset_params, files=shape_post) 
    
# FOR ALL OTHER REQUESTS THAT DO NOT UTILIZED AN UPLOADED POLYGON FILE, USE A GET REQUEST INSTEAD OF POST:
#     request = session.get(base_url, params=request_params)
    
    print('Request HTTP response: ', request.status_code)

# Raise bad request: Loop will stop for bad response code.
    request.raise_for_status()
#    print('Order request URL: ', request.url)
    esir_root = ET.fromstring(request.content)
#    print('Order request response XML content: ', request.content)

# Look up order ID
    orderlist = []   
    for order in esir_root.findall("./order/"):
        orderlist.append(order.text)
    orderID = orderlist[0]
    print('order ID: ', orderID)

# Create status URL
    statusURL = base_url + '/' + orderID
    print('status URL: ', statusURL)

# Find order status
    request_response = session.get(statusURL)    
    print('HTTP response from order response URL: ', request_response.status_code)
    
# Raise bad request: Loop will stop for bad response code.
    request_response.raise_for_status()
    request_root = ET.fromstring(request_response.content)
    statuslist = []
    for status in request_root.findall("./requestStatus/"):
        statuslist.append(status.text)
    status = statuslist[0]
    print('Data request ', page_val, ' is submitting...')
    print('Initial request status is ', status)

# Continue to loop while request is still processing
    while status == 'pending' or status == 'processing': 
        print('Status is not complete. Trying again.')
        time.sleep(10)
        loop_response = session.get(statusURL)

# Raise bad request: Loop will stop for bad response code.
        loop_response.raise_for_status()
        loop_root = ET.fromstring(loop_response.content)

# Find status
        statuslist = []
        for status in loop_root.findall("./requestStatus/"):
            statuslist.append(status.text)
        status = statuslist[0]
        print('Retry request status is: ', status)
        if status == 'pending' or status == 'processing':
            continue

# Order can either complete, complete_with_errors, or fail:
# Provide complete_with_errors error message:
    if status == 'complete_with_errors' or status == 'failed':
        messagelist = []
        for message in loop_root.findall("./processInfo/"):
            messagelist.append(message.text)
        print('error messages:')
        pprint.pprint(messagelist)

# Download zipped order if status is complete or complete_with_errors
    if status == 'complete' or status == 'complete_with_errors':
        downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/' + orderID + '.zip'
        print('Zip download URL: ', downloadURL)
        print('Beginning download of zipped output...')
        zip_response = session.get(downloadURL)
        # Raise bad request: Loop will stop for bad response code.
        zip_response.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
            z.extractall(path)
        print('Data request', page_val, 'is complete.')
    else: print('Request failed.')

#Clean up Outputs folder by removing individual granule folders 

for root, dirs, files in os.walk(path, topdown=False):
    for file in files:
        try:
            shutil.move(os.path.join(root, file), path)
        except OSError:
            pass
        
for root, dirs, files in os.walk(path):
    for name in dirs:
        os.rmdir(os.path.join(root, name))

#List files
sorted(os.listdir(path))

001
2018-10-14T00:00:00Z,2018-10-16T23:59:59Z
/home/jovyan/crackup/shapefiles/edgeworth_simple-polygon.kml
https://n5eil02u.ecs.nsidc.org/egi/capabilities/ATL06.001.xml
[{'id': 'ICESAT2', 'spatialSubsetting': 'true', 'spatialSubsettingShapefile': 'true', 'temporalSubsetting': 'true', 'type': 'both', 'maxGransSyncRequest': '100', 'maxGransAsyncRequest': '2000'}]
2018-10-14T00:00:00,2018-10-16T23:59:59
1
Order:  1
Request HTTP response:  201
order ID:  5000000320283
status URL:  https://n5eil02u.ecs.nsidc.org/egi/request/5000000320283
HTTP response from order response URL:  201
Data request  1  is submitting...
Initial request status is  processing
Status is not complete. Trying again.
Retry request status is:  complete
Zip download URL:  https://n5eil02u.ecs.nsidc.org/esir/5000000320283.zip
Beginning download of zipped output...
Data request 1 is complete.


['processed_ATL03_20190206021109_06050210_001_01.h5',
 'processed_ATL03_20190206145852_06130212_001_01.h5',
 'processed_ATL06_20181015194309_02620112_001_01.h5']